<a href="https://colab.research.google.com/github/zeekx/build_dl_framework/blob/dev/steps/step14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backward: vector, x and itself

x = Variable(np.array(1.0))
y = add(x, x)
y.backward()
print("y=x+x=2x, f'(1)=2, but x.grad=", x.grad)



In [11]:
import numpy as np

In [12]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None


    def set_creator(self, func):
        self.creator = func


# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self):
        fs = [self.creator]
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        while fs:
            f = fs.pop()
            ygs = [o.grad for o in f.outputs]
            xgs = f.backward(*ygs)
            if not isinstance(xgs, tuple):
                xgs = (xgs,)
    
            for x, g in zip(f.inputs, xgs):
                if x.grad is None:
                    x.grad = g
                else:
                    x.grad = x.grad + g
                if x.creator is not None:
                    fs.append(x.creator)


In [13]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [14]:
class Function:
    def __call__(self, *inputs): # input:[x0, x1, ...]@Variable -> [y0, y1, ...]@Variable
        self.inputs = inputs
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #unwrap
        if not isinstance(ys, tuple):
            ys = (ys,)

        self.outputs = [Variable(as_array(y)) for y in ys]
        for output in self.outputs:
            output.set_creator(self)

        return self.outputs if len(self.outputs) > 1 else self.outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [15]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx


In [16]:
class Add(Function):
    def forward(self, a, b):
        y = a + b
        return (y,)
    
    def backward(self, gy):
        return gy, gy

In [17]:
class Identical(Function):
    def forward(self, x):
        return x
    
    def backward(self, gy):
        return gy # ??? 1 or gy

In [18]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

def add(x0, x1):
  return Add()(x0, x1)

def identical(x):
  return Identical()(x)

In [19]:

x = Variable(np.array(1.0))
y = add(x, x)
y.backward()
print("y=x+x=2x, f'(1)=2, and we get x.grad=", x.grad)



y=x+x=2x, f'(1)=2, but x.grad= 2.0
